In [1]:
import os
import textblob
import tweepy
import re
import numpy as np
import pandas as pd
from textblob import TextBlob
from twitter_credentials import *

In [2]:
#API setup
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth)
    return api

In [3]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analyze_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [4]:
# create an extractor object:
extractor = twitter_setup()

# create a tweet list as follows:
tweets = extractor.search("$XRP", count = 10)
print(type(tweets))
# create a pandas dataframe as follows:
data = pd.DataFrame(data=[tweet.created_at for tweet in tweets], columns=['Date'])
# add relevant data:
data['Tweets'] = np.array([tweet.text for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])

<class 'tweepy.models.SearchResults'>


In [5]:
# create a column with the result of the analysis:

data['SA'] = np.array([ analyze_sentiment(tweet) for tweet in data['Tweets'] ])

# We display the updated dataframe with the new column:
display(data.head(10))

,Date,Tweets,Likes,RTs,SA
0,2018-02-17 18:32:37,RT @CryptoMcKenna: Projects I don't like and w...,0,6,-1
1,2018-02-17 18:32:30,RT @Ripple: We spoke with @Hodor7777 to learn ...,0,690,0
2,2018-02-17 18:32:24,Most anticipated ICO of 2018!! This will be HU...,0,0,1
3,2018-02-17 18:32:21,RT @cryptotoppick: Retweet For A Chance To Win...,0,37,1
4,2018-02-17 18:31:56,"don't miss out on registering on Binance, befo...",0,0,0
5,2018-02-17 18:31:53,RT @OTC_Bitcoin: For noobs that don't understa...,0,7,0
6,2018-02-17 18:31:53,Trading signals. Join now:\n\nhttps://t.co/jDt...,0,0,0
7,2018-02-17 18:31:52,"Upcoin Exchange looks great, 1000 Trading pair...",0,0,1
8,2018-02-17 18:31:31,"RT @PibeGeek: Cuando $XRP arranque, va a ser i...",0,6,0
9,2018-02-17 18:31:06,Me too. I issued some FUN. Just for fun. Becau...,1,0,1
